In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import commonallplayers, playerindex


bbref_DIR = "../data/bbref/"

player_dict = get_players_pbp()
teams_dict, teams_list = get_teams()

box_DIR = "../data/box/"
shot_DIR = "../data/ShotLocationData/"
fig_DIR = "../figs/teams/"

In [ ]:
gpt, gp = [],[]
seasons = range(2000,2024)
for year in range(2000,2024):
    df1 = pd.read_parquet(bbref_DIR +f"NBA_bbref_P_Adv_{year}.parquet")
    df1 = df1.query("minutes > 500")
    df1 = df1[["idPlayerNBA","ratioBPM"]]
    df1.columns = ["PLAYER_ID","BPM"]
    df1 = df1.sort_values("BPM",ascending=False)
    df1 = df1.head(25)
    players = df1["PLAYER_ID"].to_list()
    df2 = pd.read_parquet(shot_DIR +f"NBA_Shot_Details_{year}.parquet")
    df2 = df2[df2["PLAYER_ID"].isin(players)]
    df2["Season"] = year + 1
    gp.append(df2)
df3 = pd.concat(gp)

In [ ]:
df3["SHOT_ZONE_BASIC"].unique()

In [ ]:
df3["SHOT_TYPE"].unique()

In [ ]:
df3["SHOT_ZONE_RANGE"].unique()

In [ ]:
df3.columns

In [ ]:
df4 = df3.query("SHOT_TYPE == '2PT Field Goal' &  SHOT_ZONE_BASIC == 'Mid-Range'")

In [ ]:
df5 = df4.groupby(["PLAYER_NAME","Season","SHOT_ZONE_RANGE"])["SHOT_MADE_FLAG"].agg("count")
df5 = df4.groupby(["Season","SHOT_ZONE_RANGE"])["SHOT_MADE_FLAG"].agg("count")

In [ ]:
df6  = df5.reset_index()
df6 = df6.rename(columns={"SHOT_MADE_FLAG":"Shots","SHOT_ZONE_RANGE":"Shot Range"})

In [ ]:
data = df6

In [ ]:
p = (
    ggplot(df6)
    + aes(x="Season",y="Shots",color="Shot Range")
    + geom_line(size=1.5)
    + theme_idv
    + labs(
        title = "Mid Range Shots by Top 25 Players in BPM",
        subtitle = "Since 2000",
        caption="bsky:@sradjoker.cc | x:@sradjoker | source: nba.com/stats",
    )
    + theme(
        figure_size=(8,5),
    )
)
p